In [1]:
!pip install fiona
!pip install pyogrio
import pandas as pd
import os
current_directory = os.getcwd()
print(current_directory)

/Users/mac


In [2]:
import pandas as pd

# Load data from an Excel file
url2 = "https://github.com/equipounopsNICARAGUA/SEER-2030-V1/raw/main/predictions_urban.xlsx"
predictions_urban = pd.read_excel(url2)

# Access and display the DataFrame
print(predictions_urban)


         country  Year  Predicted_URBAN_SDG_7_1_1
0    Afghanistan  2022                 102.456364
1    Afghanistan  2023                 103.789091
2    Afghanistan  2024                 105.121818
3    Afghanistan  2025                 106.454545
4    Afghanistan  2026                 107.787273
..           ...   ...                        ...
814       Zambia  2026                  68.873636
815       Zambia  2027                  69.792727
816       Zambia  2028                  70.711818
817       Zambia  2029                  71.630909
818       Zambia  2030                  72.550000

[819 rows x 3 columns]


In [3]:
def apply_dynamic_correction(df, Year_col, Predicted_URBAN_SDG_7_1_1_col, target_Year, target_Predicted_URBAN_SDG_7_1_1, predictions_urban):
    try:
        # Sort the DataFrame by the year column in ascending order
        df = df.sort_values(by=[Year_col, 'country'])

        # Initialize a new column to store the correction factors
        df['Correction_Factor'] = 0

        # Calculate the correction factors for each country separately
        unique_countries = df['country'].unique()
        for country in unique_countries:
            country_mask = df['country'] == country
            min_year = df.loc[country_mask, Year_col].min()
            max_year = df.loc[country_mask, Year_col].max()

            for year in range(min_year, max_year):
                current_Predicted_URBAN_SDG_7_1_1 = df.loc[country_mask & (df[Year_col] == year), Predicted_URBAN_SDG_7_1_1_col].values[0]
                num_years_to_target = target_Year - year

                if num_years_to_target == 0 and year == target_Year:
                    initial_correction_factor = ((target_Predicted_URBAN_SDG_7_1_1 * 100) / target_Predicted_URBAN_SDG_7_1_1) - target_Predicted_URBAN_SDG_7_1_1
                    df.loc[country_mask & (df[Year_col] == year), 'Corrected_' + Predicted_URBAN_SDG_7_1_1_col] = target_Predicted_URBAN_SDG_7_1_1
                else:
                    if num_years_to_target != 0:
                        initial_correction_factor = (target_Predicted_URBAN_SDG_7_1_1 - current_Predicted_URBAN_SDG_7_1_1) / num_years_to_target
                
                df.loc[country_mask & (df[Year_col] == year), 'Correction_Factor'] = initial_correction_factor
                df.loc[country_mask & (df[Year_col] == year), 'Corrected_' + Predicted_URBAN_SDG_7_1_1_col] = current_Predicted_URBAN_SDG_7_1_1 + ((initial_correction_factor * current_Predicted_URBAN_SDG_7_1_1 / 100))

                if year < target_Year:
                    next_year = year + 1
                    num_years_to_target = target_Year - next_year
                    if num_years_to_target == 0:
                        if year == target_Year:
                            initial_correction_factor = ((target_Predicted_URBAN_SDG_7_1_1 * 100) / target_Predicted_URBAN_SDG_7_1_1) - target_Predicted_URBAN_SDG_7_1_1
                            df.loc[country_mask & (df[Year_col] == year), 'Corrected_' + Predicted_URBAN_SDG_7_1_1_col] = target_Predicted_URBAN_SDG_7_1_1
                    else:
                        if num_years_to_target != 0:
                            initial_correction_factor = (target_Predicted_URBAN_SDG_7_1_1 - df.loc[country_mask & (df[Year_col] == year), 'Corrected_' + Predicted_URBAN_SDG_7_1_1_col].values[0]) / num_years_to_target
                    
                    df.loc[country_mask & (df[Year_col] == next_year), 'Corrected_' + Predicted_URBAN_SDG_7_1_1_col] = df.loc[country_mask & (df[Year_col] == year), 'Corrected_' + Predicted_URBAN_SDG_7_1_1_col].values[0] + ((initial_correction_factor * df.loc[country_mask & (df[Year_col] == year), 'Corrected_' + Predicted_URBAN_SDG_7_1_1_col].values[0] / 100))

        return df
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return df

# Usage:
predictions_urban = apply_dynamic_correction(predictions_urban, 'Year', 'Predicted_URBAN_SDG_7_1_1', 2030, 100, predictions_urban)






















































In [4]:
predictions_urban

,country,Year,Predicted_URBAN_SDG_7_1_1,Correction_Factor,Corrected_Predicted_URBAN_SDG_7_1_1
0,Afghanistan,2022,102.456364,-0.307045,102.141776
81,Angola,2022,104.594545,-0.574318,103.993840
117,Bangladesh,2022,97.894545,0.263182,98.152186
144,Belize,2022,99.200909,0.099886,99.299997
270,Benin,2022,75.389091,3.076364,77.708333
...,...,...,...,...,...
800,United Republic of Tanzania,2030,70.295455,0.000000,118.404815
557,Vanuatu,2030,95.000000,0.000000,105.228878
35,Western Africa,2030,83.550000,0.000000,113.487626
818,Zambia,2030,72.550000,0.000000,118.037878


In [5]:
# Sort the DataFrame by 'country' in ascending order and then by 'year' in ascending order
predictions_urban = predictions_urban.sort_values(by=['country', 'Year'], ascending=[True, True])

# Display the sorted DataFrame
print(predictions_urban)



         country  Year  Predicted_URBAN_SDG_7_1_1  Correction_Factor  \
0    Afghanistan  2022                 102.456364          -0.307045   
1    Afghanistan  2023                 103.789091          -0.541299   
2    Afghanistan  2024                 105.121818          -0.853636   
3    Afghanistan  2025                 106.454545          -1.290909   
4    Afghanistan  2026                 107.787273          -1.946818   
..           ...   ...                        ...                ...   
697     Zimbabwe  2026                  91.625455           2.093636   
698     Zimbabwe  2027                  91.878182           2.707273   
699     Zimbabwe  2028                  92.130909           3.934545   
700     Zimbabwe  2029                  92.383636           7.616364   
701     Zimbabwe  2030                  92.636364           0.000000   

     Corrected_Predicted_URBAN_SDG_7_1_1  
0                             102.141776  
1                             103.227282  
2     

In [6]:
predictions_urban

,country,Year,Predicted_URBAN_SDG_7_1_1,Correction_Factor,Corrected_Predicted_URBAN_SDG_7_1_1
0,Afghanistan,2022,102.456364,-0.307045,102.141776
1,Afghanistan,2023,103.789091,-0.541299,103.227282
2,Afghanistan,2024,105.121818,-0.853636,104.224460
3,Afghanistan,2025,106.454545,-1.290909,105.080314
4,Afghanistan,2026,107.787273,-1.946818,105.688851
...,...,...,...,...,...
697,Zimbabwe,2026,91.625455,2.093636,93.543758
698,Zimbabwe,2027,91.878182,2.707273,94.365575
699,Zimbabwe,2028,92.130909,3.934545,95.755842
700,Zimbabwe,2029,92.383636,7.616364,99.419910


In [7]:
import pandas as pd

# Set the display options to show all rows
pd.set_option('display.max_rows', None)

# Display DataFrame
print(predictions_urban)



                                      country  Year  \
0                                 Afghanistan  2022   
1                                 Afghanistan  2023   
2                                 Afghanistan  2024   
3                                 Afghanistan  2025   
4                                 Afghanistan  2026   
5                                 Afghanistan  2027   
6                                 Afghanistan  2028   
7                                 Afghanistan  2029   
8                                 Afghanistan  2030   
81                                     Angola  2022   
82                                     Angola  2023   
83                                     Angola  2024   
84                                     Angola  2025   
85                                     Angola  2026   
86                                     Angola  2027   
87                                     Angola  2028   
88                                     Angola  2029   
89        

In [8]:
predictions_urban

,country,Year,Predicted_URBAN_SDG_7_1_1,Correction_Factor,Corrected_Predicted_URBAN_SDG_7_1_1
0,Afghanistan,2022,102.456364,-0.307045,102.141776
1,Afghanistan,2023,103.789091,-0.541299,103.227282
2,Afghanistan,2024,105.121818,-0.853636,104.224460
3,Afghanistan,2025,106.454545,-1.290909,105.080314
4,Afghanistan,2026,107.787273,-1.946818,105.688851
5,Afghanistan,2027,109.120000,-3.040000,105.802752
6,Afghanistan,2028,110.452727,-5.226364,104.680066
7,Afghanistan,2029,111.785455,-11.785455,98.611031
8,Afghanistan,2030,113.118182,0.000000,86.989272
81,Angola,2022,104.594545,-0.574318,103.993840


In [10]:
import pandas as pd

# Save the DataFrame as a CSV file
predictions_urban.to_csv('corrections_urban.csv', index=False)